In [79]:
import requests
import time
import json
import geojson

In [80]:
with open("route1.json", "r") as r:
    route = geojson.load(r)

In [81]:
msg = {"name": "Entity_ONE"}

In [ ]:
while True:
    for lon, lat in route["features"][0]["geometry"]["coordinates"]:
        msg["lat"] = lat
        msg["lon"] = lon
        requests.post("http://127.0.0.1:8002/producer/geostream", json=msg)
        time.sleep(1)

In [87]:
for lon, lat in route["features"][0]["geometry"]["coordinates"]:
    msg["lat"] = lat
    msg["lon"] = lon
    requests.post("http://127.0.0.1:8002/producer/geostream", json=msg)
    time.sleep(0.2)

In [49]:
response = requests.get("http://127.0.0.1:8003/ping")
response

<Response [200]>